In [ ]:
import os

from weco_datascience.reporting import get_recent_data

In [ ]:
df = get_recent_data(config=os.environ, n=100000, index="metrics-conversion-prod")


After I wrote this code, I decided to filter the imported data to exclude changes to item views deployed during week of 7/11/22. I can't get this code that I borrowed from completeness.ipynb to work though. The code works fine if I run the 2 cells above and omit the cell below.

In [ ]:
import os

from weco_datascience import reporting

df = reporting.query_es(
    config=os.environ,
    index="metrics-conversion-prod",
    query={
        "size": 100000,
        "sort": [{"@timestamp": {"order": "desc"}}],
        "query": {
            "bool": {
                "must": [
                    {"range": {"@timestamp": {"lte": "2022-11-07T00:00:00.000Z"}}},
                    {
                        "term": {
                            "page.name": {
                                "value": "work",
                            }
                        }
                    }
                ],
            }
        },
    },
)


In [ ]:
print(len(df))

What is the date range?

In [ ]:
df.head(1)

In [ ]:
df.tail(1)

Sort df by anonymousId and reindex

In [ ]:
sorted_df=df.sort_values(["anonymousId", "@timestamp"])
sorted_df2=sorted_df.reset_index(drop=True)

In [ ]:
sorted_df2.head()

1.Calculate the number of times a user clicks on the Library catalogue tab directly followed by a click on the Images tab.

In [ ]:
anonId_prev=str(36)
worksSearch2imagesSearch=0

for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"])!=anonId_prev) and ((sorted_df2.loc[i, "page.path"])=="/works"):
        anonId_prev=sorted_df2.loc[i, "anonymousId"]

    elif (sorted_df2.loc[i, "anonymousId"]==anonId_prev) and ((sorted_df2.loc[i, "page.path"])=='/images'):
        worksSearch2imagesSearch+=1
        anonId_prev==""


1b.Calculate the number of times a user clicks on the Library catalgoe tab directly followed by when a 
user clicks on the Images tab and the total number of times a user clicks on either the Images of Library catalogue tabs.

In [ ]:
anonId_prev=str(36)
imagesSearch2worksSearch=0

for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"])!=anonId_prev) and ((sorted_df2.loc[i, "page.path"])=="/images"):
        anonId_prev=sorted_df2.loc[i, "anonymousId"]

    elif (sorted_df2.loc[i, "anonymousId"]==anonId_prev) and ((sorted_df2.loc[i, "page.path"])=='/works'):
        imagesSearch2worksSearch+=1
        anonId_prev==""

tab_clicks=df.loc[(df["page.path"].isin(["/works","/images"]))]   


print("Total number of journeys between Library catalogue and Image tabs: ", worksSearch2imagesSearch+imagesSearch2worksSearch)
print("Total number of Library catalogue and Image tab clicks: ", (len(tab_clicks)) )
print("Percentage of journeys between Library catalogue and Image tabs by the total number of tab clicks: ", "{:.0%}".format(
(worksSearch2imagesSearch+imagesSearch2worksSearch)/len(tab_clicks)))


2.Calculate the percentage of works page views directly followed by 
item page views out of the total number of works page views

In [ ]:
anonId_prev=str(36)
worksPageview2itemPageview=0

for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="work" or sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i,"page.query.id"]!="")):
        anonId_prev=sorted_df2.loc[i, "anonymousId"]

    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="images" or sorted_df2.loc[i, "page.name"]=="item") and (
        sorted_df2.loc[i,"page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]!="viewer/paginator")):
        worksPageview2itemPageview+=1
        anonId_prev==""

worksPageview = int()        
worksPageview = sorted_df2.loc[
    (sorted_df2["page.name"].isin(["images", "work"]))
    & (sorted_df2["page.query.id"].notnull())
]     

print("Total number of works pageviews directly followed by item pageview: ", worksPageview2itemPageview)
print("Total number of works pageviews: ", (len(worksPageview)) )
print("Percentage of works pageviews directly followed by images search view of total works search views: ", "{:.0%}".format(
    worksPageview2itemPageview/len(worksPageview)))


3.Calculate the percentage of item pageviews directly followed by pagination usage out of the total number of item pageviews.

Note: images don't offer more than 1 page to view, so this analysis isfor catalogue items only.

In [ ]:
anonId_prev=str(36)
itemPageview2pagination=0


for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="item") and (
        sorted_df2.loc[i,"page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]!="viewer/paginator")):
        anonId_prev=sorted_df2.loc[i, "anonymousId"]

    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="item") and (
        sorted_df2.loc[i,"page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]=="viewer/paginator")):
        itemPageview2pagination+=1
        anonId_prev==""

itemPageview = sorted_df2.loc[(sorted_df2["page.name"]=="item") & (
        sorted_df2["page.pathname"]=="/item")]

print("Total number of non-image item pageviews directly followed by item pageview by pagination: ", itemPageview2pagination)
print("Total number of non-image item pageviews: ", (len(itemPageview)) )
print("Percentage of non-image item pageviews directly followed by item pageview by pagination: ", "{:.0%}".format(
    itemPageview2pagination/len(itemPageview)))


4.Calculate the percentage of searches followed by item page views followed by new search of total searches

In [ ]:
anonId_prev=str(36)
search2item=str(3)
search2item2search=0


for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="works" or sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i,"page.query.query"]!="")):
        anonId_prev=sorted_df2.loc[i, "anonymousId"]


    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="images" or sorted_df2.loc[i, "page.name"]=="item") and (
        sorted_df2.loc[i,"page.pathname"]=="/item")):
        search2item="yes"

        
    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="works" or sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i,"page.query.query"]!="") and (
        search2item=="yes")):
        search2item2search+=1
        search2item=="no"
        anonId_prev==""
       
                    
searches = sorted_df2.loc[
    (sorted_df2["page.name"].isin(["images", "work"]))
    & (sorted_df2["page.query.id"].notnull())
]     

print("Total number of searches directly followed by item pageview and then a new search: ", search2item2search)
print("Total number of searches: ", (len(searches)) )
print("Percentage of searches directly followed by item pageview and then a new search of total searches: ", "{:.0%}".format(
    search2item2search/len(searches)))

5.Calculate the percentage of works page views directly followed by a new search

In [ ]:
anonId_prev=str(36)
worksPageview2newSearch=0

for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="work" or sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i,"page.query.id"]!="")):
        anonId_prev=sorted_df2.loc[i, "anonymousId"]

    elif (((sorted_df2.loc[i, "anonymousId"])==anonId_prev) and ((sorted_df2.loc[i, "page.name"])=="works" or (
        sorted_df2.loc[i, "page.name"])=='images') and (
        (sorted_df2.loc[i, "page.query.query"])!="")):
        worksPageview2newSearch+=1
        anonId_prev==""


print("Total number of works pageviews followed by new search: ", worksPageview2newSearch)
print("Total number of works pageviews: ", len(worksPageview))
print("Percentage of new searches of total searches: ", "{:.0%}".format(worksPageview2newSearch/len(worksPageview)))
